In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("walmart").getOrCreate()
df = spark.read.csv('walmart_stock.csv',header=True,inferSchema=True)
df.columns
df.printSchema()
df.head(5)
df.describe().show()
df.describe().printSchema()
from pyspark.sql.functions import format_number
result = df.describe()
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()
df2 = df.withColumn("HV Ratio",df["High"]/df["Volume"])#.show()
df2.select('HV Ratio').show()
df.orderBy(df["High"].desc()).head(1)[0][0]
from pyspark.sql.functions import mean
df.select(mean("Close")).show()
from pyspark.sql.functions import max,min
df.select(max("Volume"),min("Volume")).show()
df.filter("Close < 60").count()
df.filter(df['Close'] < 60).count()
from pyspark.sql.functions import count
result = df.filter(df['Close'] < 60)
result.select(count('Close')).show()
(df.filter(df["High"]>80).count()*1.0/df.count())*100
from pyspark.sql.functions import corr
df.select(corr("High","Volume")).show()
from pyspark.sql.functions import year
yeardf = df.withColumn("Year",year(df["Date"]))
max_df = yeardf.groupBy('Year').max()
max_df.select('Year','max(High)').show() 
from pyspark.sql.functions import month
monthdf = df.withColumn("Month",month("Date"))
monthavgs = monthdf.select("Month","Close").groupBy("Month").mean()
monthavgs.select("Month","avg(Close)").orderBy('Month').show()